In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [4]:
df_patients_22 = pd.read_csv("VAERS/2021VAERSData/2021VAERSDATA.csv", index_col='VAERS_ID', encoding='latin1', low_memory=False)

In [5]:
df_patients_22.index
df_patients_22

,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
VAERS_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/28/2020,12/30/2020,2.0,None,PVT,NaN,None,None,None,NaN,NaN,2,01/01/2021,NaN,Y,NaN,Pcn and bee venom
916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/31/2020,12/31/2020,0.0,NaN,SEN,NaN,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,NaN,NaN,2,01/01/2021,NaN,Y,NaN,"""Dairy"""
916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,12/31/2020,12/31/2020,0.0,NaN,SEN,NaN,None,None,None,NaN,NaN,2,01/01/2021,NaN,NaN,Y,Shellfish
916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/23/2020,12/23/2020,0.0,none,WRK,NaN,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","got measles from measel shot, mums from mumps ...",NaN,2,01/01/2021,NaN,NaN,NaN,"Diclofenac, novacaine, lidocaine, pickles, tom..."
916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,12/22/2020,12/29/2020,7.0,NaN,PUB,NaN,Na,Na,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,Na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997120,12/31/2021,ID,56.0,56.0,NaN,M,NaN,I had symptoms of low level un-diagnosed kidn...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,11/28/2021,11/30/2021,2.0,lab test for UTI,UNK,NaN,none,un-diagnosed kidney infection,none,NaN,NaN,2,12/31/2021,NaN,Y,NaN,none
1997121,12/31/2021,OH,69.0,69.0,NaN,M,NaN,I got very bad eczema after first shot. Then i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,02/26/2021,03/04/2021,6.0,went to my doctor he said I have eczema. Gave...,PHM,NaN,b12 and d3 vitamins,NaN,hypothyroid,NaN,NaN,2,12/31/2021,NaN,NaN,NaN,"penicillin, statin drugs, codeine"
1997122,12/31/2021,NV,50.0,50.0,NaN,M,NaN,Hives all over the body and swelling of the li...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,09/04/2021,10/01/2021,27.0,NaN,PHM,NaN,NaN,None,None,NaN,NaN,2,12/31/2021,NaN,NaN,NaN,None


In [6]:
df_vax_22 = pd.read_csv("VAERS/2021VAERSData/2021VAERSVAX.csv", index_col='VAERS_ID', encoding='latin1')

In [7]:
df_vax_22

,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
VAERS_ID,,,,,,,
916600,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
916601,COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
916602,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916603,COVID19,MODERNA,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
916604,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
...,...,...,...,...,...,...,...
2013746,HIBV,MERCK & CO. INC.,T026960,UNK,IM,LL,HIB (PEDVAXHIB)
2013746,PNC13,PFIZER\WYETH,DR1779,UNK,IM,LL,PNEUMO (PREVNAR13)
2013746,RV5,MERCK & CO. INC.,1742458,UNK,PO,MO,ROTAVIRUS (ROTATEQ)


In [8]:
df_COVID19_22 = df_vax_22[df_vax_22["VAX_TYPE"] == "COVID19"]

In [9]:
df_COVID19_22

,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
VAERS_ID,,,,,,,
916600,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
916601,COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
916602,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916603,COVID19,MODERNA,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
916604,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
...,...,...,...,...,...,...,...
2005264,COVID19,PFIZER\BIONTECH,EN6208,UNK,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
2005274,COVID19,MODERNA,028L20A,UNK,IM,LA,COVID19 (COVID19 (MODERNA))
2005274,COVID19,MODERNA,014M20A,UNK,IM,LA,COVID19 (COVID19 (MODERNA))


In [10]:
df_patients_COVID19 = pd.merge(df_patients_22, df_COVID19_22, left_index=True, right_index=True)

In [11]:
df_patients_22.shape, df_COVID19_22.shape, df_patients_COVID19.shape

((749067, 34), (737971, 7), (737971, 41))

In [12]:
# pd.set_option('display.max_rows', 100)
df_patients_COVID19

,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
VAERS_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/28/2020,12/30/2020,2.0,None,PVT,NaN,None,None,None,NaN,NaN,2,01/01/2021,NaN,Y,NaN,Pcn and bee venom,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/31/2020,12/31/2020,0.0,NaN,SEN,NaN,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,NaN,NaN,2,01/01/2021,NaN,Y,NaN,"""Dairy""",COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,12/31/2020,12/31/2020,0.0,NaN,SEN,NaN,None,None,None,NaN,NaN,2,01/01/2021,NaN,NaN,Y,Shellfish,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/23/2020,12/23/2020,0.0,none,WRK,NaN,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","got measles from measel shot, mums from mumps ...",NaN,2,01/01/2021,NaN,NaN,NaN,"Diclofenac, novacaine, lidocaine, pickles, tom...",COVID19,MODERNA,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,12/22/2020,12/29/2020,7.0,NaN,PUB,NaN,Na,Na,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,Na,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005264,04/24/2021,HI,NaN,NaN,NaN,M,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,PVT,NaN,NaN,NaN,NaN,NaN,NaN,2,04/24/2021,NaN,NaN,NaN,NaN,COVID19,PFIZER\BIONTECH,EN6208,UNK,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
2005274,04/26/2021,NC,65.0,65.0,NaN,F,NaN,New onset nasal herpes simplex coming up about...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/01/2021,03/10/2021,9.0,NaN,PVT,NaN,NaN,NaN,NaN,NaN,NaN,2,04/24/2021,NaN,Y,NaN,NaN,COVID19,MODERNA,028L20A,UNK,IM,LA,COVID19 (COVID19 (MODERNA))
2005274,04/26/2021,NC,65.0,65.0,NaN,F,NaN,New onset nasal herpes simplex coming up about...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/01/2021,03/10/2021,9.0,NaN,PVT,NaN,NaN,NaN,NaN,NaN,NaN,2,04/24/2021,NaN,Y,NaN,NaN,COVID19,MODERNA,014M20A,UNK,IM,LA,COVID19 (COVID19 (MODERNA))


In [13]:
df_PFIZER_BIONTECH = df_patients_COVID19[df_patients_COVID19["VAX_MANU"] == "MODERNA"]

In [14]:
df_PFIZER_BIONTECH.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339877 entries, 916600 to 2009884
Data columns (total 41 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   RECVDATE         339877 non-null  object 
 1   STATE            302148 non-null  object 
 2   AGE_YRS          309475 non-null  float64
 3   CAGE_YR          302392 non-null  float64
 4   CAGE_MO          561 non-null     float64
 5   SEX              339877 non-null  object 
 6   RPT_DATE         271 non-null     object 
 7   SYMPTOM_TEXT     339774 non-null  object 
 8   DIED             4694 non-null    object 
 9   DATEDIED         4392 non-null    object 
 10  L_THREAT         4753 non-null    object 
 11  ER_VISIT         26 non-null      object 
 12  HOSPITAL         20841 non-null   object 
 13  HOSPDAYS         13803 non-null   float64
 14  X_STAY           152 non-null     object 
 15  DISABLE          4875 non-null    object 
 16  RECOVD           308614 non-null

In [15]:
# table = pd.pivot_table(df_PFIZER_BIONTECH, values='DIED', columns=['DIED'], aggfunc=np.sum)

In [16]:
df = df_PFIZER_BIONTECH[df_PFIZER_BIONTECH['DIED']=='Y'][['VAX_LOT']]
df

,VAX_LOT
VAERS_ID,
917117,NaN
917790,NaN
917793,NaN
918065,025J20-2A
918487,011L2Oa
...,...
1995984,NaN
1996537,037F21A
1996542,NaN


In [17]:
# df = df.groupby(['VAX_LOT']).count()
#df2 = df.sort_values(by=['DIED'], ascending=False)
#pd.set_option('display.max_rows', None)
#print(df2)
pd.set_option('display.max_rows', None)
df.value_counts()

VAX_LOT        
039K20A            93
012L20A            72
010M20A            69
013L20A            68
030L20A            54
037K20A            53
016M20A            53
012M20A            53
029L20A            51
024M20A            51
025L20A            50
022M20A            50
027L20A            48
010A21A            48
013M20A            45
026A21A            45
031L20A            45
015M20A            45
007M20A            44
026L20A            42
031M20A            40
013A21A            39
011A21A            39
032L20A            38
004M20A            38
030A21A            38
011J20A            37
011M20A            37
028L20A            35
011L20A            35
041L20A            35
025J20-2A          32
Unknown            32
030M20A            31
012A21A            31
032H20A            31
040A21A            30
032M20A            30
043L20A            30
042L20A            29
006M20A            29
023M20A            29
002A21A            27
017B21A            26
036A21A         